Launch interactive version: 👉👉👉 [![Try ``dyce``](https://jupyterlite.readthedocs.io/en/latest/_static/badge.svg)](https://posita.github.io/dyce-notebooks/lab?path=one-bookshelf%2Fbookmark-no-hp-rpg-370430%2Fbookmark_no_hp_rpg.ipynb) 👈👈👈 *[[source](https://github.com/posita/dyce-notebooks/tree/main/notebooks/one-bookshelf/bookmark-no-hp-rpg-370430)]*

## [``dyce``](https://posita.github.io/dyce/) computation of mechanic odds for [“Bookmark No HP RPG”](https://www.drivethrurpg.com/product/370430/Bookmark-No-HP-RPG)

Once viewing this notebook in Jupyter Lab, select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        import anydyce
    except (ImportError, ModuleNotFoundError):
        # See <https://jupyterlite.readthedocs.io/en/stable/howto/configure/simple_extensions.html#avoid-the-drift-of-versions-between-the-frontend-extension-and-the-python-package>
        requirements = ["anydyce~=0.4.4"]
        try:
            import piplite ; await piplite.install(requirements, keep_going=True)
            # Work around <https://github.com/jupyterlite/jupyterlite/issues/838>
            import matplotlib.pyplot ; matplotlib.pyplot.clf()
        except ImportError:
            import pip ; pip.main(["install"] + requirements)
    import anydyce

In [2]:
from dyce import H
from dyce.evaluation import aggregate_weighted
from functools import cache

def bookmark__no_hp_rpg(n: int, d: H, flip_negatives: bool = False) -> H:
    failure_outcome = min(d)
    assert failure_outcome >= 0, f"lowest value is {failure_outcome}; negative values are reserved for tracking degrees of failure"
    failures_appearing = n @ d.eq(failure_outcome)
    d_without_failure = d.draw(failure_outcome).lowest_terms()

    def _sub_hs_from_failures_gen():
        for num_failures, count in failures_appearing.items():
            if num_failures > 0:
                if flip_negatives:
                    failures = -_min_from_nd_dp(n - num_failures, d_without_failure)
                    failures = failures.draw({0: failures.get(0, 0), -max(d) - 1: -failures.get(0, 0)})
                    yield failures, count
                else:
                    yield -_max_from_nd_dp(n - num_failures, d_without_failure), count
            else:
                yield _max_from_nd_dp(n, d_without_failure), count
 
    return aggregate_weighted(_sub_hs_from_failures_gen())

def _max_from_nd_dp(n: int, d: H) -> H:
    return _val_from_nd_dp(n, d, max)

def _min_from_nd_dp(n: int, d: H) -> H:
    return _val_from_nd_dp(n, d, min)

@cache
def _val_from_nd_dp(n: int, d: H, cmp) -> H:
    if n == 0:
        return H({0: 1})
    elif n == 1:
        return d
    elif n > 1:
        return d.map(cmp, _val_from_nd_dp(n - 1, d, cmp))
    else:
        assert False, "shouldn't ever be here"

### Interpreting results

*Bookmark ~HP~ RPG* affords degrees of success and failure. If a one appears at least once when rolling, the result is a failure, the degree of which depends on the highest die that is not a one. When rolling a d6 with a difficulty of five, if the roll is (2, 1, 3, 5, 2), the result would be a failure with a degree of 5. If the roll is (2, 3, 2, 2, 3), the result would be a success with a degree of 3.

The graphs below show the likelihood of a particular result given all possible rolls. Degrees of failure are represented by negative numbers and degrees of success are represented by positive numbers. By default, a zero indicates a roll constituting solely ones, e.g., (1, 1, 1, 1, 1). Expanding on our previous example, a roll of (2, 1, 3, 5, 1) is represented as -5 (i.e., a failure whose maximum non-one value is 5).

Because the values in each graph are sorted least-to-greatest, the worst results (most severe degree of failure, or least impactful degree of success) hover around zero (with zero being the worst possible outcome for the player), and fan out into better (less severe, more impactful) results from there. What&rsquo;s interesting in the extremes is that the mechanic favors either least severe failures or more impactful successes, with the worst outcomes being very unlikely.

If ``Invert Failures`` is enabled, lower negative values (i.e., those ***farther*** from zero) are more severe failures, the most negative value being the worst (i.e., all ones). This is basically akin to looking to the ***minimum*** non-one outcome in the case of a failure and treating that as the least severe. The negative of one more than the highest die face (e.g., -7 on a d6) represents an all-one roll. This is a little counter-intuitive, but orders results worst-to-best for the player.

In [3]:
from anydyce import jupyter_visualize
from ipywidgets import widgets

dice = {
    "d4": H(4),
    "d6": H(6), 
    "d8": H(8),
    "d10": H(10),
    "d12": H(12),
}

def _display(
    selected_dice: list[str],
    difficulties: tuple[int, int],
    flip_failures: bool,
):
    difficulty_lo, difficulty_hi = difficulties
    jupyter_visualize(
        [
            (
                f"{d_str} results\nvs. difficulty of {difficulty}",
                bookmark__no_hp_rpg(difficulty, dice[d_str], flip_failures),
            )
            for difficulty in range(difficulty_lo, difficulty_hi + 1) for d_str in selected_dice
        ],
        # controls_expanded=True,
        initial_burst_color_bg_trnsp=True,
        initial_burst_columns=len(selected_dice),
        initial_enable_cutoff=False,
        initial_resolution=3 * (len(selected_dice) + 1),
    )

dice_keys = list(dice.keys())
dice_widget = widgets.SelectMultiple(
    options=dice_keys,
    value=dice_keys[0:-1],
    description="Dice",
)

difficulties_widget = widgets.IntRangeSlider(
    value=(2, 6),
    min=1,
    max=10,
    step=1,
    continuous_update=False,
    description="Difficulties",
)

flip_failures_widget = widgets.Checkbox(
    value=False,
    description="Invert Failures",
)

display(widgets.HBox([
    dice_widget,
    widgets.VBox([
        difficulties_widget,
        flip_failures_widget,
    ]),
]))

display(
    widgets.interactive_output(
        _display,
        {
            "selected_dice": dice_widget,
            "difficulties": difficulties_widget,
            "flip_failures": flip_failures_widget,
        },
    )
)

Output()

Code below is used to validate (test) that above.

In [4]:
from functools import reduce
from itertools import repeat

def bookmark_no_hp_rpg_opaque_failures(n: int, d: H) -> H:
    failure_outcome = min(d)
    assert failure_outcome >= 0, f"lowest value is {failure_outcome}; negative values are reserved for tracking degrees of failure"
    no_failures = (n @ d.eq(failure_outcome)).eq(0)
    d_without_failure = d.draw(failure_outcome).lowest_terms()
    max_success = _max_from_nd(n, d_without_failure)
    return H(
        (outcome, count)
        for outcome, count in (no_failures * max_success).items()
    )

def _max_from_nd(n: int, d: H) -> H:
    return reduce(
        lambda lh, rh: lh.map(max, rh),
        repeat(d, n),
        H({0: 1}),
    )

for difficulty in range(1, 11):
    for die in dice.values():
        assert _max_from_nd(difficulty, die) == _max_from_nd_dp(difficulty, die)
        failures_collapsed = H(
            (0 if outcome < 0 else outcome, count)
            for outcome, count in bookmark__no_hp_rpg(difficulty, die).items()
        )
        assert bookmark_no_hp_rpg_opaque_failures(difficulty, die) == failures_collapsed